In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
from tqdm import tqdm

In [49]:
df=pd.read_csv('top10000_review_preprocess.csv',encoding='utf-8')
df

,user_id,level,user_info,hash_id,review_date,product_id,product_brand,product_name,product_option,review_text,star_rating,helpful_count,사이즈,밝기,색감,두께감,count
0,명료한한강나시,8,NaN,9447214fa2649ab776bedb7667c1fdfb,2023-09-09,3085109,반스,라 코스타 슬라이드 온 - (체커보드) 블랙:마쉬멜로우 / VN0A5HFER6R1,245,배송 이 빨라 좋아요 당일 출고 되 네요 감사합니다,5,0,보통이에요,보통이에요,보통이에요,NaN,4619
1,명료한한강나시,8,NaN,9447214fa2649ab776bedb7667c1fdfb,2023-10-06,3018563,캉골,듀오 캔버스 버켓백 Ⅱ 3912 아이보리,ONE,배송도 빠르고 가방동 마음에 들어요 잘쓰겠습니다 ^^,5,0,보통이에요,보통이에요,보통이에요,NaN,4619
2,명료한한강나시,8,NaN,9447214fa2649ab776bedb7667c1fdfb,2024-02-26,963689,쌤소나이트 레드,ELIUN BACKPACK M_BLACK AU709002,FREE,배송빨라요정사이즈에요착용감좋아요편안해요,5,0,보통이에요,보통이에요,보통이에요,NaN,4619
3,명료한한강나시,8,NaN,9447214fa2649ab776bedb7667c1fdfb,2024-02-26,1271098,내셔널지오그래픽,N205AFW900 다이브 플러스 빅로고 슬리퍼 MINT,MINT/230,배송빨라요정사이즈에요착용감좋아요편안해요,5,1,보통이에요,보통이에요,보통이에요,NaN,4619
4,명료한한강나시,8,남성 · 170cm · 70kg,9447214fa2649ab776bedb7667c1fdfb,2024-02-26,1272812,와릿이즌,와펜 가디건 블랙,L,배송빨라요정사이즈에요착용감좋아요편안해요,5,0,보통이에요,보통이에요,보통이에요,보통이에요,4619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368354,Sun아아아,7,NaN,691c9beca008d69ec949be11bbaacf15,2024-07-10,3109573,블루엘리펀트,CAPELLA black-orange tint,none,무신사 이벤트때 착한가격으로 잘 구매했어요감사합니다,5,0,보통이에요,보통이에요,보통이에요,NaN,5
1368355,Sun아아아,7,NaN,691c9beca008d69ec949be11bbaacf15,2024-07-10,3290281,아디다스,버뮤다 - 핑크 / GY7386,230,안녕하세요버뮤다 핑크 이쁘네요잘 구매했어요감사합니다,5,0,보통이에요,보통이에요,보통이에요,NaN,5
1368356,평화빼기하나,8,NaN,507abcb707af0d765b0ab5216fa382f3,2024-01-06,3454614,보테가베네타,남성 인트레치아토 레더 지갑 - 블랙 / 222338V46511000,FREE,무신사에서 좋은제품 사게되어서 좋습니다.다음에도 살게요.,4,0,보통이에요,NaN,NaN,NaN,3
1368357,평화빼기하나,8,남성 · 173cm · 65kg,507abcb707af0d765b0ab5216fa382f3,2024-01-18,3189553,생로랑,남성 울 테디 재킷 - 블랙 / 354718Y180W1070,52,생로랑핏에 맞게 세쿠시하게 입기보다는 과잠처럼 여유있게 입는게 좋아서 일주러 크게 ...,5,4,보통이에요,보통이에요,보통이에요,보통이에요,3


In [37]:
Q1 = df['count'].quantile(0.25)
Q3 = df['count'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1
upper_bound = Q3

filtered_df = df[(df['count'] > lower_bound) & (df['count'] <= upper_bound)].reset_index()
filtered_df

,user_id,level,user_info,hash_id,review_date,product_id,product_brand,product_name,product_option,review_text,star_rating,helpful_count,사이즈,밝기,색감,두께감,count
339773,FaRiNnI,6,NaN,978caf7dc274e65fee9c301118213170,2022-07-05,2043588,노매뉴얼,DOODLE LOGO HOODIE - BLACK,M,핏도 너무 이쁘고 스트릿룩에 찰떡이네요 잘 입겠습니다,5,0,보통이에요,어두워요,선명해요,보통이에요,223
339774,FaRiNnI,6,NaN,978caf7dc274e65fee9c301118213170,2022-07-09,2496111,비전스트릿웨어,VSW Sleepy Logo T-Shirts Charcoal,L,진짜 그래픽 반팔 맛집이네요 친구 선물로 줬습니다,5,0,보통이에요,보통이에요,보통이에요,보통이에요,223
339775,FaRiNnI,6,NaN,978caf7dc274e65fee9c301118213170,2022-07-09,2508796,예일,[ONEMILE WEAR] 2PACK BIG OXFORD SS SHIRT WHITE...,M,너무 이뻐요 여름 필수템인 것 같아요 선물로 잘 줬습니다,5,0,보통이에요,보통이에요,보통이에요,보통이에요,223
339776,FaRiNnI,6,NaN,978caf7dc274e65fee9c301118213170,2022-07-22,2043588,노매뉴얼,DOODLE LOGO HOODIE - BLACK,M,생각보다 크지 않아서 오히려 좋았어요 시보리도 짱짱해요,5,0,보통이에요,어두워요,선명해요,보통이에요,223
339777,FaRiNnI,6,NaN,978caf7dc274e65fee9c301118213170,2022-07-23,1736562,제로,Deep One Tuck Sweat Pants [White Grey],M,생각보다 무릎이 잘 늘어나지만 핏이 너무 이뻐요,5,0,커요,밝아요,선명해요,보통이에요,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022619,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,996180,무신사 스탠다드,릴렉스 핏 크루 넥 반팔 티셔츠 [네이비],M,너무 두껍지도 않고 얇지도 않는 두께감이고 소매 접어서 입으면 더 예쁩니다,5,0,보통이에요,보통이에요,보통이에요,보통이에요,105
1022620,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,1551840,제로,Deep One Tuck Sweat Pants [Grey],M / 여름용,mesh소재라 각 잡힌 느낌은 아니고 편하게 툭 걸치기 좋아요 면에 비해 시원하긴 합니다,5,3,보통이에요,보통이에요,보통이에요,얇아요,105
1022621,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,2411265,무신사 스탠다드,쿨탠다드 릴렉스 핏 크루 넥 반팔 티셔츠 [블랙],M,쿨탠다드라 시원하긴 한데 땀이나면 좀 잘 보이는 단점이 있네요,5,0,보통이에요,보통이에요,보통이에요,얇아요,105
1022622,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,2445081,51퍼센트,METAL Logo tshirt - BLACK,M,무지반팔에 메탈로고로 딱 포인트 들어가서 너무 심심하지 않네요,5,0,보통이에요,보통이에요,보통이에요,보통이에요,105


In [75]:
grouped_reviews = filtered_df.groupby('user_id')['review_text'].apply(list).reset_index()
grouped_reviews

,user_id,review_text
0,!_OoO~!,"[색감이 너무 이뻐서 잘입을 것 같아요질도 좋고 핏도 좋아요 감사합니다, 가격에 비..."
1,!이자까야!,[이거 진짜 이쁘고 따뜻하고 가성비 미쳤어욥남성복이라서 고민했지만 너무 이쁘니까 잘...
2,#J0120,"[기장 좋고 소재는 빠삭빠삭한 재질이라서 한번 세탁하고 입으면 될 거 같아요, 고양..."
3,#준민,"[좋아요 잘 입고 있어요 데일리로 쓰기 편하고 좋아요, 좋아요 잘 입고 있어요 데일..."
4,#탐탐이,"[마크도 귀엽고 면이 두껍고 탄탄해서 아주 좋습니다!, 블루 색상 딱 원하던 색이라..."
...,...,...
4678,힐군,"[역시 양말도 무신사죠쫀쫀하고 잘 안늘어나요 추천, 생각했던 재질은 아닌데 가성비로..."
4679,힙텍쳌,"[가격대비 퀄 진짜 조아요 만족합니다 다른색도 사야겟어요 ㅋ, 만족해요 진짜 싸게 ..."
4680,힙한블루캡모자,"[짙은 색감이나 사이즈 및 핏 모두 대단히 만족스럽네요~, 색감과 디자인 모두 기대..."
4681,힙한쿠바힙합,[두껍고 뻗뻗한감이 있지만 맘에들어요 그런데 쵸크자국이 있던데 지워서 입었어요 자주...


In [79]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from tqdm import tqdm

# Function to preprocess text
def preprocess(text, okt):
    tokens = okt.morphs(text, stem=True)
    return ' '.join(tokens)

# Function to check for redundancy
def check_redundancy(cosine_similarities, threshold=0.8):
    num_reviews = len(cosine_similarities)
    # Checking if any pair of reviews has cosine similarity greater than the threshold
    for i in range(num_reviews):
        for j in range(i + 1, num_reviews):
            if cosine_similarities[i, j] > threshold:
                return True
    return False

# Function to determine appropriateness
def is_user_appropriate(reviews, okt):
    # Preprocess reviews
    processed_reviews = [preprocess(review, okt) for review in reviews]
    
    # Vectorization
    vectorizer = TfidfVectorizer(max_features=50000)
    tfidf_matrix = vectorizer.fit_transform(processed_reviews)
    
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(tfidf_matrix)
    
    # Check redundancy
    return not check_redundancy(cosine_similarities)

# Initialize the tokenizer once
okt = Okt()


# 사용자 적합성 평가
true_user = []

for i in tqdm(range(len(grouped_reviews))):
    reviews = grouped_reviews.iloc[i, 1]
    result = is_user_appropriate(reviews, okt)
    if result:
        true_user.append(grouped_reviews.iloc[i, 0])

print(len(true_user))

100%|██████████████████████████████████████████████████████████████████████████████| 4683/4683 [59:07<00:00,  1.32it/s]

551


In [99]:
final_user_reviews=filtered_df[filtered_df['user_id'].isin(true_user)].reset_index()
final_user_reviews.drop(columns='index',inplace=True)
final_user_reviews

,user_id,level,user_info,hash_id,review_date,product_id,product_brand,product_name,product_option,review_text,star_rating,helpful_count,사이즈,밝기,색감,두께감,count
0,goldeoah,7,NaN,eefbea2c3ad5685401e64be3d5983693,2023-01-25,1639228,무신사 스탠다드,[힛탠다드] 우먼즈 타이즈_이너 웨어 [라이트 베이지],L,안입은 것 같고 따듯해요퀄리티도 꽤 괜찮은것 같아요,5,0,보통이에요,보통이에요,보통이에요,NaN,223
1,goldeoah,7,여성 · 160cm · 53kg,eefbea2c3ad5685401e64be3d5983693,2023-01-25,2126170,무신사 스탠다드 우먼,우먼즈 기모 스웨트 팬츠 [오트밀],S,생각보다 무게감은 가벼워서 따뜻할까 걱정했는데롱패딩이랑 입으면 괜찮아요,4,0,보통이에요,밝아요,보통이에요,보통이에요,223
2,goldeoah,7,NaN,eefbea2c3ad5685401e64be3d5983693,2023-01-25,2157040,나른,미스티 맨살 트렁크 3팩_NWTRA03216_3SET,M,겨울철 여유있는 하의랑 입기 딱이에요별로 티 안남,5,0,보통이에요,보통이에요,보통이에요,NaN,223
3,goldeoah,7,NaN,eefbea2c3ad5685401e64be3d5983693,2023-01-25,2199164,폴로 랄프 로렌,폴로 베어 리브 비니 - 네이비,FREE,귀엽구 따듯해요푹 눌러쓰면 따듯하지만 골무같아오.,5,0,보통이에요,보통이에요,보통이에요,NaN,223
4,goldeoah,7,NaN,eefbea2c3ad5685401e64be3d5983693,2023-01-25,2336381,에스쁘아,테이핑 컨실러 롱웨어,아이보리_611123167,마스크 때문에 화장 잘 안해서 잡티만 커버하려고 샀는데 괨찮은덧 같아요,5,0,NaN,NaN,NaN,NaN,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79032,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,996180,무신사 스탠다드,릴렉스 핏 크루 넥 반팔 티셔츠 [네이비],M,너무 두껍지도 않고 얇지도 않는 두께감이고 소매 접어서 입으면 더 예쁩니다,5,0,보통이에요,보통이에요,보통이에요,보통이에요,105
79033,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,1551840,제로,Deep One Tuck Sweat Pants [Grey],M / 여름용,mesh소재라 각 잡힌 느낌은 아니고 편하게 툭 걸치기 좋아요 면에 비해 시원하긴 합니다,5,3,보통이에요,보통이에요,보통이에요,얇아요,105
79034,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,2411265,무신사 스탠다드,쿨탠다드 릴렉스 핏 크루 넥 반팔 티셔츠 [블랙],M,쿨탠다드라 시원하긴 한데 땀이나면 좀 잘 보이는 단점이 있네요,5,0,보통이에요,보통이에요,보통이에요,얇아요,105
79035,황금 돼지,6,남성 · 177cm · 63kg,e58e1c74bbe04e00692f524eb64ed4f4,2024-07-04,2445081,51퍼센트,METAL Logo tshirt - BLACK,M,무지반팔에 메탈로고로 딱 포인트 들어가서 너무 심심하지 않네요,5,0,보통이에요,보통이에요,보통이에요,보통이에요,105


In [105]:
final_user_reviews.to_csv('final_user_reviews(551).csv',encoding='utf-8')